# Fluidinámica computacional (Sesión 01)

<img src="figures/LogoCinvestav.png" style="width:330px;height:100px" title="Cinvestav">

**CINVESTAV Unidad Saltillo**

**Programa de Maestría en Ciencias en Ingeniería Metalúrgica**

Dr. Edgar Ivan Castro Cedeño


Enero - Junio 2025

Contacto:
[edgar.castro@cinvestav.mx](mailto:edgar.castro@cinvestav.mx)

# 4. Cálculo de transporte de un escalar

## 4.1 Preparación del cálculo

Antes de efectuar el cálculo se deben de copiar algunos archivos desde el directorio **`tundishFlow2D`** al directorio **`tundishScalar2D`**:

- Copiar la malla desde `tundishFlow2D/system/blockMeshDict` hasta `tundishScalar2D/system/blockMeshDict`.

- Copiar modificaciones en la topología de la malla desde `tundishFlow2D/system/topoSetDict` hasta `tundishScalar2D/system/topoSetDict`.

- Copiar instrucciones de parchado de la malla desde `tundishFlow2D/system/createPatchDict` hasta `tundishScalar2D/system/createPatchDict`.

- Copiar campo de velocidades calculado desde `tundishFlow2D/180/U` hasta `tundishScalar2D/0/U`.

    - A notar que el nombre del directorio `tundishFlow2D/180` podría ser diferente, de acuerdo al tiempo que se haya dejado correr el cálculo.


## 4.2 Instrucciones a ejecutar en la terminal

<div style="background: lightgray">

```sh
$ blockMesh
$ checkMesh
$ topoSet
$ createPatch -overwrite
$ scalarTransportFoam
```

</div>

<center>

***Bloque 01. Instrucciones para cálcular el transporte del trazador.***

</center>

**Los primeros cuatro comando en el bloque de instrucciones se utilizan para realizar operaciones relacionadas con la especificación de la geometría del dominio de cálculo:**

- `blockMesh`: crear malla del dominio, de acuerdo a lo especificado en el archivo `blockMeshDict`.

- `checkMesh`: evaluar la calidad del mallado (un mallado apropiado es fundamental para obtener buena convergencia de la solución).

- `topoSet`: especificar las características topológicas de los parches de entrada (inlet) y salida (outlet) en el dominio, a partir de instrucciones especificadas en el archivo `topoSetDict`.

- `createPatch`: aplicar los parches de entrada (inlet) y salida (outlet) en la malla del dominio, a partir de las instrucciones especificadas en el archivo `createPatchDict`.

Los archivos con la información de la malla que se ha creado y parchado al ejecutar las instrucciones arriba, se encuentran en el directorio `constant/polymesh`.

Más información de mallado disponible en la [documentación](https://www.openfoam.com/documentation/user-guide/4-mesh-generation-and-conversion).

**El último comando en el bloque de instrucciones se utiliza para ejecutar la aplicación `scalarTransportFoam`, con la cuál se obtendrá el campo de velocidades:**

- `scalarTransportFoam` es un resolvedor para la ecuación de transporte de un escalar `T`, que requiere de un campo de velocidades "congelado" (que será el calculado previamente con `pimpleFoam`).

- El resolvedor se ejecuta de acuerdo con las instrucciones especificadas en los archivos `controlDict`, `fvSchemes`, y `fvSolution`.

- Los campos iniciales y condiciones a la frontera están contenidos en los archivos dentro del directorio `0`.

- Las propiedades de transporte están contenidas dentro de los archivos `transportProperties`.



## 4.3 Criterio para terminación del cálculo

Para obtener un estimado del periodo de tiempo físico para el que se debe de simular el transporte de un escalar, es conveniente efectuar un estimado del tiempo de residencia promedio de un paquete de fluido en el sistema.

$$
t_r = \frac{\bar{V}}{\dot{v}} = \frac{0.28 \, [\mathrm{m^3}]}{0.001 \, [\mathrm{m^3.s^{-1}}]} = 280 \, [s].
$$

$$
\bar{V} = (3.5 \times 0.8 \times 0.1) \, [\mathrm{m^3}]
$$

$$
\dot{v} = (0.1 \times 0.1) \, [\mathrm{m^2}] \times 0.1 \, [\mathrm{m.s^{-1}}]
$$

Basándose en este estimado, se considera un tiempo de simulación de $540$ $[\mathrm{s}]$, que es equivalente a aproximadamente 2 tiempos de residencia promedio en el reactor.

La manera de operar en este tutorial consiste en utilizar un resolvedor en régimen transitorio, y dejar correr la simulación suficiente tiempo como para obtener un campo de velocidades pseudo-estacionario.

- Esto es, el comportamiento promedio del flujo es prácticamente constante. Pueden existir pequeñas fluctuaciones que "desaparecen" al efectuar el promedio de los campos.

- Esto se verifica efectuando balances de masa y trazando perfiles de velocidad en el reactor.

    - En el archivo `controlDict`, dentro del diccionario `functions` hay un diccionario que da instrucciones al resolvedor para monitorear el valor del escalar `T` a la salida del distribuidor..

    - Al correr el caso, los archivos con los resultados de "monitoreo" se guardan en el directorio `postProcessing`.

## 4.3 Post-procesamiento

El postratamiento de los resultados obtenidos se lleva a cabo utilizando [paraFoam](https://www.openfoam.com/documentation/user-guide/7-post-processing/7.1-parafoam), que es un script que permite procesar resultados de OpenFoam con el software [ParaView](https://www.paraview.org/). 

Como se muestra en el listado debajo, para ejecutar este software se utiliza el comando paraFoam, y el signo &, indica a la terminal que el proceso se va a ejecutar en segundo plano.

<div style="background: lightgray">

```sh
$ paraFoam -vtk &
```

</div>

<center>

***Bloque 02. Instrucciones para ejecutar software de postprocesamiento.***

</center>

En ocasiones, cuando se trabaja con el sistema operativo Windows, puede haber dificultades para visualizar los resultados, en estos casos se lleva a cabo una exportación de los resultados de `OpenFoam` al formato [VTK](https://vtk.org/), para que los resultados puedan directamente ser abiertos con el software [ParaView](https://www.paraview.org/). 

<div style="background: lightgray">

```sh
$ foamToVTK
```

</div>

<center>

***Bloque 03. Instrucciones para exportar resultados en formato VTK.***

</center>

<center>

<img src="figures/images/paraViewScalarTransportFoam2D.png" style="width:600px" title="Paraview">

***Figura 01. Ejemplo de post-procesamiento en Paraview***

</center>